In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

#### 默认情况下，工具通过检查函数签名来推断参数模式。对于更严格的需求，可以指定自定义输入模式，以及自定义验证逻辑。
#### 在Python函数中，函数签名（function signature）指的是函数的声明部分，包括函数名、参数列表和返回值类型。函数签名描述了函数的输入和输出，帮助开发者理解函数的用途和使用方式。

In [ ]:
from typing import Any, Dict
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.tools.requests.tool import RequestsGetTool,TextRequestsWrapper
from pydantic import BaseModel, Field, root_validator

In [ ]:
llm = OpenAI(temperature=0)

In [1]:
import tldextract #tldextract是一个第三方模块,意思就是Top Level Domain extract，即顶级域名提取

_APPROVED_DOMAINS = {
    "langchain",
    "wikipedia",
}


class ToolInputSchema(BaseModel):
    url: str = Field(...)

    @root_validator
    def validate_query(cls, values: Dict[str, Any]) -> Dict:
        url = values["url"]
        domain = tldextract.extract(url).domain
        if domain not in _APPROVED_DOMAINS:
            raise ValueError(
                f"Domain {domain} is not on the approved list:"
                f" {sorted(_APPROVED_DOMAINS)}"
            )
        return values


tool = RequestsGetTool(
    args_schema=ToolInputSchema, requests_wrapper=TextRequestsWrapper()
)

NameError: name 'BaseModel' is not defined

In [ ]:
agent = initialize_agent(
    [tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

In [ ]:
# This will succeed, since there aren't any arguments that will be triggered during validation
answer = agent.run("What's the main title on langchain.com?")
print(answer)

In [ ]:
agent.run("What's the main title on google.com?")

In [ ]:
answer = agent.run("What's the main title on wikipedia.com?")
print(answer)